In [1]:
import warnings
import pandas as pd
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import json

warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pandas'

In [3]:
# First we get days were games were played

import requests
from bs4 import BeautifulSoup as bs
import re
import datetime as dt
import pickle
import os

In [29]:
# we build a table of games, summary from each date
# build a loop in python that scrapes the website https://baseballsavant.mlb.com/gamefeed?date={date} using beautifulsoup
# the date argument is a datetime object that needs to be converted to string, the main element to be scaped is a 'div' element with id='charts

all_dates = []
for date in range(2018, 2024):
    url = f'https://www.baseball-reference.com/leagues/MLB/{date}-schedule.shtml'
    resp = requests.get(url, verify=False)
    days = re.findall("<h3>(.*{year})</h3>".format(year=date), resp.text)
    dates = [dt.datetime.strptime(d, "%A, %B %d, %Y") for d in days]
    all_dates.append(dates)

date_str = all_dates[5][0].strftime('%Y-%m-%d')
data = []
url = f'https://baseballsavant.mlb.com/gamefeed?date={date_str}'
response = requests.get(url, verify=False)
# Parse the HTML content with BeautifulSoup
soup = bs(response.text)
# Find the 'div' element with id='charts'
charts = soup.findAll('div', {'name': '718608'})

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True
# options.add_argument(('--headless'))
options.add_argument("--windows-size=1920,1200")

DRIVER_PATH = 'C:/Users/andres.moreno/Downloads/111/chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)



In [128]:
summaries = pd.DataFrame(columns=['date', 'game_id', 'away_score', 'home_score', 'away_team', 'home_team'])
for year in all_dates:
    for date in year:

        date_str = date.strftime('%Y-%m-%d')
        if date_str > '2023-04-12':
            continue
        url = f'https://baseballsavant.mlb.com/gamefeed?date={date_str}'
        driver.get(url)
        try:
            elem = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, "game-container step")))
        except:
            pass
        soup = bs(driver.page_source)
        games = soup.findAll('div', {'class': 'game-container step'})

        for game in games:
            scores = game.findAll('div', {'class': 'game-column score'})
            cities = game.findAll('div', {'class': 'team-name'})
            game_id = game.find('div', {'class': 'game-meta-container'})['name']
            summaries.loc[len(summaries)] = [date_str, game_id, scores[0].text.strip(), scores[1].text.strip(),
                                             cities[0].text.strip(),
                                             cities[1].text.strip()]



In [132]:
summaries.away_score = pd.to_numeric(summaries['away_score'], errors='coerce', downcast='float')
summaries.home_score = pd.to_numeric(summaries['home_score'], errors='coerce', downcast='float')


In [130]:
summaries.away_score.mean()

4.483914375305176

In [133]:
summaries

,date,game_id,away_score,home_score,away_team,home_team
0,2018-03-29,529407,8.0,4.0,Cubs,Marlins
1,2018-03-29,529419,4.0,9.0,Cardinals,Mets
2,2018-03-29,529414,2.0,3.0,Twins,Orioles
3,2018-03-29,529411,4.0,1.0,Astros,Rangers
4,2018-03-29,529415,6.0,1.0,Yankees,Blue Jays
...,...,...,...,...,...,...
11324,2023-04-12,718600,7.0,9.0,Red Sox,Rays
11325,2023-04-12,718603,3.0,4.0,Tigers,Blue Jays
11326,2023-04-12,718601,4.0,5.0,Reds,Braves
11327,2023-04-12,718595,10.0,1.0,Royals,Rangers


In [134]:
# summaries.to_pickle('summaries.pkl')

In [5]:
summaries = pickle.load(open('summaries.pkl', 'rb'))

,date,game_id,away_score,home_score,away_team,home_team
0,2018-03-29,529407,8.0,4.0,Cubs,Marlins
1,2018-03-29,529419,4.0,9.0,Cardinals,Mets
2,2018-03-29,529414,2.0,3.0,Twins,Orioles
3,2018-03-29,529411,4.0,1.0,Astros,Rangers
4,2018-03-29,529415,6.0,1.0,Yankees,Blue Jays
...,...,...,...,...,...,...
11324,2023-04-12,718600,7.0,9.0,Red Sox,Rays
11325,2023-04-12,718603,3.0,4.0,Tigers,Blue Jays
11326,2023-04-12,718601,4.0,5.0,Reds,Braves
11327,2023-04-12,718595,10.0,1.0,Royals,Rangers


In [6]:
def get_pitcher_data(js, where):
    """Return a list of dictionaries containing the pitching statistics for a specified team.

    Args:
        js (dict): A JSON object containing the data to be processed.
        where (str): A string representing the team whose data is to be retrieved.

    Returns:
        list: A list of dictionaries containing the pitching statistics for each pitcher on the specified team.
    """

    pitcher_ids = js['boxscore']['teams'][where]['pitchers']
    pitchers = []

    players = js['boxscore']['teams'][where]['players']
    for player_id, values in players.items():
        player_id = values['person']['id']
        if player_id in pitcher_ids:
            p_stats = js['boxscore']['teams'][where]['players']['ID' + str(player_id)]['stats']['pitching']
            p_id = js['boxscore']['teams'][where]['players']['ID' + str(player_id)]['person']
            p_stats['is_home_team'] = True if where is 'home' else False
            p_stats.update(p_id)
            pitchers.append(p_stats)

    return pitchers



In [7]:
def getr_game_data(pk):
    """
    Retrieves game data from either a saved JSON file or the MLB Baseball Savant API
    and returns a dictionary of the relevant game data.

    Args:
        pk (int): A unique identifier for the game.

    Returns:
        dict: A dictionary containing the game data for the specified game.
    """
    # Construct filename and filepath from game identifier
    filename = str(pk) + '.json'
    filepath = os.path.join('gamejson', filename)

    # Check if data for the game is already saved as a JSON file
    if os.path.exists(filepath):
        # If the data is saved, load it from the JSON file
        with open(filepath, 'r') as f:
            js = json.load(f)
    else:
        # If the data is not saved, fetch it from the MLB Baseball Savant API
        pay_url = 'https://baseballsavant.mlb.com/gf?game_pk=%s' % pk
        resp = requests.get(pay_url, verify=False)
        js = json.loads(resp.content)

    try:
        # Extract relevant game data from JSON object
        home_score = js['scoreboard']['linescore']['teams']['home']['runs']
        away_score = js['scoreboard']['linescore']['teams']['away']['runs']
        home_team = js['home_team_data']['abbreviation']
        away_team = js['away_team_data']['abbreviation']
        game_id = pk
        date = js['game_date']
        away_batting = js['boxscore']['teams']['away']['teamStats']['batting']
        away_pitching = js['boxscore']['teams']['away']['teamStats']['pitching']
        away_fielding = js['boxscore']['teams']['away']['teamStats']['fielding']
        home_batting = js['boxscore']['teams']['home']['teamStats']['batting']
        home_pitching = js['boxscore']['teams']['home']['teamStats']['pitching']
        home_fielding = js['boxscore']['teams']['home']['teamStats']['fielding']

        # Retrieve individual pitcher data for both teams using the get_pitcher_data function
        away_pitchers = get_pitcher_data(js, 'away')
        home_pitchers = get_pitcher_data(js, 'home')
    except Exception as e:
        print('Game not processed: ', pk, 'error:', e)
        return pd.DataFrame()

    # Save JSON data to file
    with open('gamejson/' + str(pk) + '.json', 'w') as f:
        json.dump(js, f)

    # Create dictionary containing relevant game data
    game_summary = {
        'game': game_id,
        'date': date,
        'home_team': home_team,
        'away_team': away_team,
        'home_score': home_score,
        'away_score': away_score
    }
    data = {
        'game_summary': game_summary,
        'home_batting': home_batting,
        'away_batting': away_batting,
        'home_pitching': home_pitching,
        'away_pitching': away_pitching,
        'home_fielding': home_fielding,
        'away_fielding': away_fielding,
        'home_pitchers': home_pitchers,
        'away_pitchers': away_pitchers
    }

    return data

def get_df_game_summary(game_info):
    game_df = pd.DataFrame([game_info])
    return game_df

def get_df_team_performance(game_info, what):
    where = what[:4]
    is_home_team = True if where == 'home' else False
    if isinstance(game_info[what], list):
        team_df = pd.DataFrame(game_info[what])
    else:
        team_df = pd.DataFrame([game_info[what]])
    team_df['is_home_team'] = is_home_team
    team_df['team'] = game_info['game_summary'][where + '_team']
    team_df['game'] = game_info['game_summary']['game']
    return team_df

### main loop processing

In [ ]:
# def load_data(what):


In [8]:
summaries_2023 = summaries[summaries['date']>= '2023-01-01']
summaries_2023['game_id'] = summaries_2023['game_id'].astype(int)

In [19]:
batting = pd.DataFrame()
pitching = pd.DataFrame()
pitchers = pd.DataFrame()
games = pd.DataFrame()
fielding = pd.DataFrame()

num_rows = len(summaries_2023)
print(f"Initial number of rows to process: {num_rows}")
games_bk = pickle.load(open('df_games.pkl', 'rb'))
batting_bk = pickle.load(open('df_batting.pkl', 'rb'))
pitching_bk = pickle.load(open('df_pitching.pkl', 'rb'))
pitchers_bk = pickle.load(open('df_pitchers.pkl', 'rb'))
fielding_bk = pickle.load(open('df_fielding.pkl', 'rb'))
# games_bk = pickle.load(open('df_games.pkl', 'rb'))

summaries_to_process = summaries_2023[~summaries_2023['game_id'].isin(games_bk['game'])]
num_rows = len(summaries_to_process)
print(f"Not found on previous backup: {num_rows}")
start_time = time.time()
j = 0
for index, row in summaries_to_process.iterrows():
    g_id = int(row['game_id'])
    if g_id in games_bk.game.unique():
        #skip if exists
        continue
    game_data = getr_game_data(g_id)

    #check if game is valid, if not skip to next one
    try:
        games = games.append(get_df_game_summary(game_data['game_summary']))
    except:
        continue

    target_pairs = [    ['away_batting', batting],
                        ['home_batting', batting],
                        ['away_pitching', pitching],
                        ['home_pitching', pitching],
                        ['away_pitchers', pitchers],
                        ['home_pitchers', pitchers],
                        ['away_fielding', fielding],
                        ['home_fielding', fielding]
                        ]

    for i in range(len(target_pairs)):
        game_df = get_df_team_performance(game_data, target_pairs[i][0])
        target_pairs[i][1] = target_pairs[i][1].append(game_df)
        # update original DataFrame
        if i < 2:
            batting = batting.append(game_df)
        elif i < 4:
            pitching = pitching.append(game_df)
        elif i < 6:
            pitchers = pitchers.append(game_df)
        else:
            fielding = fielding.append(game_df)
    if j % 10 == 0:
        j += 1
        print(f'processing game: ', g_id)
        print(f"{j}/{num_rows} rows processed.")

end_time = time.time()
time_spent = end_time - start_time
print(f"Processing time: {time_spent:.2f} seconds.")

Initial number of rows to process: 190
Not found on previous backup: 1
Game not processed:  718700 error: 'runs'
Processing time: 0.63 seconds.


In [20]:
games = pd.concat([games_bk,games])
batting = pd.concat([batting_bk,batting])
pitching = pd.concat([pitching_bk,pitching])
pitchers = pd.concat([pitchers_bk,pitchers])
fielding = pd.concat([fielding_bk,fielding])

games.to_pickle('df_games.pkl')
batting.to_pickle('df_batting.pkl')
pitching.to_pickle('df_pitching.pkl')
pitchers.to_pickle('df_pitchers.pkl')
fielding.to_pickle('df_fielding.pkl')



### Add additional game data, columns to numeric

In [26]:
game_df = pd.DataFrame(games)
game_df['date'] = pd.to_datetime(game_df['date'], errors='coerce')
game_df['home_score'] = pd.to_numeric((game_df['home_score']), errors='coerce')
game_df['away_score'] = pd.to_numeric((game_df['away_score']), errors='coerce')

In [113]:
### batting, pitching, fielding
batting_df = pd.DataFrame(batting)
for column in batting_df.keys():
    if any(x in column for x in ['is_home_team', 'game', 'team'] ): continue
    batting_df[column] = pd.to_numeric(batting_df[column], errors='coerce', downcast='float')

pitching_df = pd.DataFrame(pitching)
for column in pitching_df.keys():
    if any(x in column for x in ['is_home_team', 'game', 'team'] ): continue
    pitching_df[column] = pd.to_numeric(pitching_df[column], errors='coerce', downcast='float')
fielding_df = pd.DataFrame(fielding)
for column in fielding_df.keys():
    if any(x in column for x in ['is_home_team', 'game', 'team'] ): continue
    fielding_df[column] = pd.to_numeric(fielding_df[column], errors='coerce', downcast='float')



In [115]:
game_df.groupby('game').agg('count').sort_values(by='game', ascending=False)

,date,home_team,away_team,home_score,away_score
game,,,,,
718782,1,1,1,1,1
718781,1,1,1,1,1
718780,1,1,1,1,1
718779,1,1,1,1,1
718778,1,1,1,1,1
...,...,...,...,...,...
718601,1,1,1,1,1
718600,1,1,1,1,1
718599,1,1,1,1,1


In [91]:
### pitching, keep only starting pitchers
# TODO: WHY NOT MOST COMMON PITCHER?
pitcher_df = pd.DataFrame(pitchers)
for column in pitcher_df.keys():
    if any(x in column for x in ['note', 'summary', 'fullName', 'id', 'link', 'team', 'game', 'is_home_team']): continue
    pitcher_df[column] = pd.to_numeric(pitcher_df[column], errors='coerce', downcast='float')

pitcher_df = pitcher_df[pitcher_df['gamesStarted']==1]
# lets remove inherited run
pitcher_df = pitcher_df.drop(columns=[x for x in pitcher_df.keys() if 'inherited' in x])
pitcher_df = pitcher_df.drop(columns=['note', 'summary', 'link'])
pitcher_df.head()

,gamesPlayed,gamesStarted,flyOuts,groundOuts,airOuts,runs,doubles,triples,homeRuns,strikeOuts,...,homeRunsPer9,catchersInterference,sacBunts,sacFlies,passedBall,is_home_team,id,fullName,team,game
0,1,1,1.0,6.0,3.0,3.0,0.0,0.0,1.0,7.0,...,1.80,0.0,0.0,0.0,0.0,False,519242,Chris Sale,BOS,718690
1,1,1,1.0,7.0,6.0,5.0,1.0,0.0,1.0,1.0,...,1.59,0.0,0.0,0.0,0.0,True,605513,Spencer Turnbull,DET,718690
1,1,1,5.0,2.0,9.0,0.0,0.0,0.0,0.0,7.0,...,0.00,0.0,0.0,0.0,0.0,False,592332,Kevin Gausman,TOR,718686
3,1,1,2.0,5.0,3.0,5.0,3.0,0.0,2.0,9.0,...,3.18,0.0,0.0,0.0,0.0,True,543475,Jordan Lyles,KC,718686
1,1,1,1.0,4.0,3.0,3.0,1.0,0.0,0.0,3.0,...,0.00,0.0,0.0,0.0,0.0,False,622072,Alex Wood,SF,718687


## Calculate Differences in the statistics

In [40]:
### we're going to calculate the average, standard deviation and skew
# for time periods 5, 10, 45,
#  grouped by team
# then shift the data
# then take the diffrence of the opposing team

In [41]:
import numpy as np

#finish the following pythong function
def add_rolling(period, df, stat_columns):
    for s in stat_columns:
        if 'object' in str(df[s].dtype): continue
        df[s+'_'+str(period)+'_Avg'] = df.groupby('team')[s].apply(lambda x:x.rolling(period).mean())
        df[s+'_'+str(period)+'_Std'] = df.groupby('team')[s].apply(lambda x:x.rolling(period).std())
        df[s+'_'+str(period)+'_Skew'] = df.groupby('team')[s].apply(lambda x:x.rolling(period).skew())
    return df

In [121]:
def get_diff_df(df, name, is_pitcher=False):
    #runs for each of the stat dataframes, returns the difference in stats

    #set up dataframe with time index
    # df['date'] = pd.to_datetime(df['game_id'].str[3:-1], format="%Y%m%d")
    df = df.sort_values(by='date').copy()
    newindex = df.groupby('date')['date'].apply(lambda x: x + np.arange(x.size).astype(np.timedelta64(1, 'D')))
    df = df.set_index(newindex).sort_index()

    # get stat columns
    stat_cols = [x for x in df.columns if 'int' in str(df[x].dtype)]
    stat_cols.extend([x for x in df.columns if 'float' in str(df[x].dtype)])

    #add lags
    df = add_rolling('5d', df, stat_cols) # this game series
    df = add_rolling('10d', df, stat_cols)
    df = add_rolling('45d', df, stat_cols)
    df = add_rolling('180d', df, stat_cols) # this season
    df = add_rolling('730d', df, stat_cols) # 2 years

    # reset stat columns to just the lags (removing the original stats)
    df.drop(columns=stat_cols, inplace=True)
    stat_cols = [x for x in df.columns if 'int' in str(df[x].dtype)]
    stat_cols.extend([x for x in df.columns if 'float' in str(df[x].dtype)])

    # shift results so that each row is  a pregame stat
    df = df.reset_index(drop=True)
    df = df.sort_values(by='date')

    for s in stat_cols:
        if is_pitcher:
            df[s] = df.groupby('name')[s].shift(1)
        else:
            df[s] = df.groupby('team')[s].shift(1)

    # calculate differences in pregame stats from home vs. away teams
    away_df = df[~df['is_home_team']].copy()
    away_df = away_df.set_index('game')
    away_df = away_df[stat_cols]

    home_df = df[df['is_home_team']].copy()
    home_df = home_df.set_index('game')
    home_df = home_df[stat_cols]
    # return home_df
    diff_df = home_df.subtract(away_df, fill_value=0)
    diff_df = diff_df.reset_index()

    # clean column names
    for s in stat_cols:
        diff_df[name + "_" + s] = diff_df[s]
        diff_df.drop(columns=s, inplace=True)
    print(diff_df.shape)
    return diff_df

df = game_df
batting_df =batting_df.drop_duplicates()
batting_df['game'].value_counts().sort_values(ascending=False)

#
# df['game'] = df['game'].astype(str)
#
#
#
# date_df = df[['game', 'date']]
# print(df.shape)
# # batting
# bt_df = batting_df.copy()
# print('bt', bt_df.shape, date_df.shape)
# bt_df['game'] = bt_df['game'].astype(str)
# print('bt', bt_df.shape, date_df.shape)
# bt_df =  bt_df.merge(date_df, how='inner', on='game')
# print('bt', bt_df.shape, date_df.shape)
# df = pd.merge(left=df, right=get_diff_df(bt_df, 'batting'), on='game', how='left')
# print('finished')

718690    2
718675    2
718672    2
718687    2
718682    2
         ..
718696    2
718695    2
718689    2
718691    2
718692    2
Name: game, Length: 185, dtype: int64

In [106]:
## merge all DFs
df = game_df
df =df.drop_duplicates()
df['game'] = df['game'].astype(str)
date_df = df[['game', 'date']]
print(df.shape)
# batting
bt_df = batting_df.copy()
print('bt', bt_df.shape, date_df.shape)
bt_df['game'] = bt_df['game'].astype(str)
print('bt', bt_df.shape, date_df.shape)
bt_df =  bt_df.merge(date_df, how='inner', on='game')
print('bt', bt_df.shape, date_df.shape)
df = pd.merge(left=df, right=get_diff_df(bt_df, 'batting'), on='game', how='left')
print(df.shape)
# # pitching
# pitching_df = pitching_df.copy()
# pitching_df['game'] = pitching_df['game'].astype(str)
# pitching_df =  pitching_df.merge(date_df, how='inner', on='game')
# df = pd.merge(left=df, right=get_diff_df(pitching_df, 'pitching'), on='game', how='left')
# print(df.shape)
# # fielding
# fielding_df = fielding_df.copy()
# fielding_df['game'] = fielding_df['game'].astype(str)
# fielding_df =  fielding_df.merge(date_df, how='inner', on='game')
# df = pd.merge(left=df, right=get_diff_df(fielding_df, 'pitching'), on='game', how='left')
# print(df.shape)
# # pitchers
# pitcher_df = pitcher_df.copy()
# pitcher_df['game'] = pitcher_df['game'].astype(str)
# pitcher_df =  pitcher_df.merge(date_df, how='inner', on='game')
#
# df = pd.merge(left=df, right=get_diff_df(pitcher_df, 'pitching'), on='game', how='left')
# print(df.shape)
# df.shape

(189, 6)
bt (378, 33) (189, 2)
bt (378, 33) (189, 2)
bt (394, 34) (189, 2)
(213, 456)


In [83]:
df.head()

,game,date,home_team,away_team,home_score,away_score,batting_flyOuts_5d_Avg,batting_flyOuts_5d_Std,batting_flyOuts_5d_Skew,batting_groundOuts_5d_Avg,...,pitching_catchersInterference_730d_Skew_y,pitching_sacBunts_730d_Avg_y,pitching_sacBunts_730d_Std_y,pitching_sacBunts_730d_Skew_y,pitching_sacFlies_730d_Avg_y,pitching_sacFlies_730d_Std_y,pitching_sacFlies_730d_Skew_y,pitching_passedBall_730d_Avg,pitching_passedBall_730d_Std,pitching_passedBall_730d_Skew
0,718690,2023-04-06,DET,BOS,3,6,-0.666667,-1.939261,-2.317634,-0.333333,...,NaN,0.0,0.000000,NaN,0.200000,0.447214,2.236068,0.0,0.0,NaN
1,718686,2023-04-06,KC,TOR,3,6,-0.250000,-0.957427,0.854563,3.000000,...,NaN,0.0,0.000000,NaN,0.000000,0.000000,NaN,0.0,0.0,NaN
2,718687,2023-04-06,CWS,SF,6,16,0.250000,-1.000718,0.752837,-1.000000,...,NaN,0.5,0.707107,NaN,-0.142857,-0.377964,-2.645751,0.0,0.0,NaN
3,718682,2023-04-06,COL,WSH,1,0,-2.666667,-1.323000,-2.667270,-0.333333,...,NaN,0.0,0.000000,NaN,0.750000,0.957427,0.854563,0.0,0.0,NaN
4,718681,2023-04-06,ATL,SD,7,6,-2.500000,-3.535534,NaN,4.500000,...,NaN,-0.1,-0.316228,-3.162278,0.600000,0.894427,1.257788,0.0,0.0,NaN


In [86]:
df.shape

(524469, 2001)